# lorapy symbol extraction

In [1]:
%load_ext autoreload
%autoreload 2

import pathlib

import os, sys
from loguru import logger
logger.remove(None)
logger.add(sys.stdout, colorize=True)

import numpy as np
from six.moves import cPickle
from tqdm import tqdm_notebook

import matplotlib.pyplot as plt 
plt.style.use('dark_background')
%matplotlib inline

In [2]:
import lorapy

## setup 

In [3]:
_BASE_DATA_DIR = pathlib.Path('/Users/trevor/Work/1-projects/lora-processing/data')

BENCHTOP_DATA_DIR = _BASE_DATA_DIR.joinpath('lora-benchtop-recording')
OTA_DATA_DIR = _BASE_DATA_DIR.joinpath('LoRa_Recording_OTA')
OUTDOOR_DATA_DIR = _BASE_DATA_DIR.joinpath('LoRa_Recording_Outdoor')
DOTP_DATA_DIR = _BASE_DATA_DIR.joinpath('symbol-ref')
SYMBOL_DIR = _BASE_DATA_DIR.joinpath('lora-symbols-20k')

# load

## dat files

In [ ]:
loader = lorapy.load_dat(BENCHTOP_DATA_DIR, autoload=True)
loader.file_list

## dotp files

In [ ]:
ploader = lorapy.load_dotp(DOTP_DATA_DIR)
ploader.file_list

## symbols

In [4]:
symloader = lorapy.load_dotp(SYMBOL_DIR)
symloader.file_list

2020-04-09 07:06:31.408 | DEBUG    | lorapy.io._base_loader:_validate_data_path:140 - set datafile directory: /Users/trevor/Work/1-projects/lora-processing/data/lora-symbols-20k
2020-04-09 07:06:31.418 | INFO     | lorapy.io._base_loader:_process_data_dir:153 - found 106 data file(s)


[DotPFile(id=0 | name='lora_BW2_SF11_915MHz_20ksps_Att60_v0.p'),
 DotPFile(id=1 | name='lora_BW8_SF12_915MHz_20ksps_Att0_v0.p'),
 DotPFile(id=2 | name='lora_BW8_SF12_915MHz_20ksps_Att20_v0.p'),
 DotPFile(id=3 | name='lora_BW1_SF11_915MHz_20ksps_Att80_v0.p'),
 DotPFile(id=4 | name='lora_BW8_SF10_915MHz_20ksps_Att80_v0.p'),
 DotPFile(id=5 | name='lora_BW7_SF10_915MHz_20ksps_Att80_v0.p'),
 DotPFile(id=6 | name='lora_BW7_SF12_915MHz_20ksps_Att20_v0.p'),
 DotPFile(id=7 | name='lora_BW2_SF11_915MHz_20ksps_Att40_v0.p'),
 DotPFile(id=8 | name='lora_BW9_SF10_915MHz_20ksps_Att0_v0.p'),
 DotPFile(id=9 | name='lora_BW7_SF10_915MHz_20ksps_Att0_v0.p'),
 DotPFile(id=10 | name='lora_BW2_SF12_915MHz_20ksps_Att120_v0.p'),
 DotPFile(id=11 | name='lora_BW8_SF12_915MHz_20ksps_Att100_v0.p'),
 DotPFile(id=12 | name='lora_BW8_SF12_915MHz_20ksps_Att40_v0.p'),
 DotPFile(id=13 | name='lora_BW7_SF12_915MHz_20ksps_Att60_v0.p'),
 DotPFile(id=14 | name='lora_BW9_SF10_915MHz_20ksps_Att100_v0.p'),
 DotPFile(id=15 | na

# process 

## symbol correlation settings

In [ ]:
_step_dict = {
    1: 100,
    2: 100,
    7: 4,
    8: 2,
    9: 2,
}

## functions

In [ ]:
def _load_matching_dotp(bw: int, sf: int):
    return ploader.filter(bw=bw, sf=sf)[0]

def _convert_files(file, dotp_file):
    return file.to_signal(), dotp_file.to_signal()

def _extract_and_manual_adjust(baseline_symbol, step_dict):
    signal.extract_packets(method='slide-mean', auto_adj=False, overlap=0.7)
    signal.adjust_packets(
        force_check=True, baseline_symbol=base_symbol, 
        range_factor=10, step=step_dict[signal.stats.bw], scalar=0.6
    )
    
    return signal.packets


def _format_output_path(packet):
    _base_path = pathlib.Path('data/lora-symbols-20k')
    filename = pathlib.Path(packet.stats.filename)
    out_path = _base_path.joinpath(filename.with_suffix('').with_suffix('.p'))
    
    return out_path

def _save_symbols(data, packet):
    out_path = _format_output_path(packet)
    
    with out_path.open('wb') as outfile:
        cPickle.dump(data, outfile)

        
def _extract_symbols(packet):
    packet.extract_preamble_symbols()
    return packet._raw_symbols

def _extract_and_save_symbols(packets):
    full_array = np.vstack([
        _extract_symbols(packet)
        for packet in packets
    ])
    
    packet = packets[0]
    _save_symbols(full_array, packet)
        

## select matching .p file

In [ ]:
exceptioned_files = []

for file in tqdm_notebook(loader.file_list):
    file.load()
    dotp_file = _load_matching_dotp(file.bw, file.sf)

    signal, base_symbol = _convert_files(file, dotp_file)
    logger.info(f'\n{signal}')
    logger.info(f'\n{base_symbol}')
    
    try:
        packets = _extract_and_manual_adjust(base_symbol, _step_dict)
        _extract_and_save_symbols(packets)
    except Exception:
        exceptioned_files.append(file.name) 
    

# viewing